In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
from qiskit_machine_learning.neural_networks import EstimatorQNN,SamplerQNN
from IPython.display import clear_output
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
from qiskit.circuit import ParameterVector
from qiskit_algorithms.utils import algorithm_globals
import time as t
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2,RealAmplitudes
import numpy as np
from qiskit.primitives import Estimator
import pickle 

In [2]:
VQE_training = False
n_qubits=4
j_coupling=0.5
g_coupling=1.5
n_rep_VQEansatz=1

def callback_graph(x,name='loss'):
    #clear_output(wait=True)
    plt.title("Training curve")
    plt.xlabel("Iteration")
    plt.ylabel(name)
    plt.plot(range(len(x)), x,label='ground state energy')
    #plt.show()


In [16]:
def getGoundState(n_qubits=n_qubits,j_coupling=j_coupling,g_coupling=g_coupling,n_rep_VQEansatz=1,nSteps=200
                 ,learning_rate=1e-1):
    h_coupling=j_coupling*g
    qr_VQE = QuantumRegister(n_qubits)
    qc_VQE = QuantumCircuit(qr_VQE)
    ansatz_VQE =  RealAmplitudes(num_qubits=n_qubits,reps=n_rep_VQEansatz)
    qc_VQE.h(qr_VQE)
    qc_VQE.compose(ansatz_VQE, inplace=True)  
    
    observable = SparsePauliOp.from_list([("X"+"I"*(n_qubits-1) , h_coupling )])
    for i in np.arange(1,n_qubits,1):
        observable=observable + SparsePauliOp.from_list([("I"*(i)+"X"+"I"*(n_qubits-1-i) , -h_coupling )])
    for i in range(n_qubits-1):
        observable=observable + (SparsePauliOp.from_list([("I"*(i)+"ZZ"+"I"*(n_qubits-2-i) ,- j_coupling)]))
        
       
    qnn_VQE=EstimatorQNN(
        estimator=Estimator(options={"shots":1e10}),
        circuit=qc_VQE,
        weight_params=ansatz_VQE.parameters,
        observables=observable
        )

    eigenValues=np.linalg.eig(observable.to_matrix())[0]
    eigenValues.sort()

    groundEnergy=eigenValues[0]
    #print("ground state energy =", groundEnergy)
    
    init_weights=np.random.rand(qnn_VQE.num_weights)*2*np.pi
    weights=init_weights
    
    loss=[]
    for i in range(nSteps):
        vev=qnn_VQE.forward([], weights)[0][0]
        input_grad, weight_grad=qnn_VQE.backward([],weights)
        grad=weight_grad[0,0,:]
        weights=weights-learning_rate*grad
        loss.append(vev)

#         if i%1==0:

#             clear_output(wait=True)
#             plt.figure(1)
#             callback_graph(loss)
#             plt.plot(range(len(loss)),groundEnergy*np.ones(len(loss)),label='truth')
#             plt.legend()
#             #plt.yscale('log')
#             plt.show()

    return weights,vev,groundEnergy

Next, prepare a series of ground states and its labels based on the value of |g|

For |g|<1, two-fold degenerate ground state.

For |g|>1, non-degenerate ground state.

In [18]:
if VQE_training:#Obtaining ground state for |g|<1 
    nSteps=50
    lowGs=np.arange(0,0.9,0.05)
    highGs=np.arange(1.1,2.0,0.05)
    w_low=[]
    Egs_estimate_low=[]
    Egs_low=[]
    for g in lowGs:
        for i in range(10):
            weights,Egs_estimate,Egs=getGoundState(g_coupling=g,nSteps=nSteps,learning_rate=1e0)
            diff=abs(Egs_estimate-Egs)/abs(Egs)

            if diff< 0.015:
                break
                
        print("g = ",g," diff=",diff)
                
        w_low.append(w_low)
        Egs_estimate_low.append(Egs_estimate)
        Egs_low.append(Egs)

    print("lowG finished")
    dict_low={"weights": w_low,"E_gs_estimate": Egs_estimate_low,
              "E_gs":Egs_low,"g":lowGs}
    with open('lowG_dict.pkl', 'wb') as f:
        pickle.dump(dict_low, f)


    w_high=[]
    Egs_estimate_high=[]
    Egs_high=[]
    for g in highGs:
        for i in range(10):
            weights,Egs_estimate,Egs=getGoundState(g_coupling=g,nSteps=nSteps)
            diff=abs(Egs_estimate-Egs)/abs(Egs)
            if diff< 0.015:
                break
        print("g = ",g," diff=",diff)
        w_high.append(w_high)
        Egs_estimate_high.append(Egs_estimate)
        Egs_high.append(Egs)

    dict_high={"weights": w_high,"E_gs_estimate":Egs_estimate_high,
               "E_gs": Egs_high ,"g":highGs}
    with open('highG_dict.pkl', 'wb') as f:
        pickle.dump(dict_high, f)

g =  0.0  diff= 4.7720745044739964e-05
g =  0.05  diff= 2.0764610829150285e-06
g =  0.1  diff= 3.412114680431665e-05
g =  0.15000000000000002  diff= 0.0001634765191005972
g =  0.2  diff= 0.0006045124482746156
g =  0.25  diff= 0.003326368617238768
g =  0.30000000000000004  diff= 0.002693163426487821
g =  0.35000000000000003  diff= 0.004098362440541296
g =  0.4  diff= 0.014975516518475323
g =  0.45  diff= 0.009580114214706972
g =  0.5  diff= 0.01315843479795649
g =  0.55  diff= 0.017209639650114877
g =  0.6000000000000001  diff= 0.019500955903741887
g =  0.65  diff= 0.014935578138387817
g =  0.7000000000000001  diff= 0.01331948347101772
g =  0.75  diff= 0.012103486551859638
g =  0.8  diff= 0.01437370275230792
g =  0.8500000000000001  diff= 0.01208471054965599
lowG finished
g =  1.1  diff= 0.09144238383413902
g =  1.1500000000000001  diff= 0.07103414581739258
g =  1.2000000000000002  diff= 0.07342111626850545
g =  1.2500000000000002  diff= 0.024113874296854867
g =  1.3000000000000003  dif

In [19]:
with open('lowG_dict.pkl', 'rb') as f:
    dict_low = pickle.load(f)

with open('highG_dict.pkl', 'rb') as f:
    dict_high = pickle.load(f)